# Reto 03-A-Grounding (Fundamentación) 

## 1. Descripción General 

La Fundamentación (Grounding) es una técnica utilizada cuando deseas que el modelo devuelva respuestas fiables a una pregunta dada. A menudo, los modelos GPT necesitarán contexto adicional para proporcionar una respuesta que no alucine, también conocida como dar respuestas falsas. Recuerda que estos modelos GPT solo han sido entrenados con datos hasta septiembre de 2021. Además, los modelos no han sido entrenados con datos específicos de casos de uso.

Existen un par de métodos para llevar a cabo la fundamentación. En este escenario, nos centraremos principalmente en el grounding básico dentro del prompt. En el reto cuatro, verás otras aplicaciones de fundamentación utilizando una base de conocimientos externa e implementando la técnica de Generación Aumentada con Recuperación, o RAG.

Para entender los conceptos básicos de la fundamentación y sus beneficios, este notebook te guiará a través de un ejemplo. A continuación se muestra el escenario que incorporaremos.

## 2. Escenario

Estás escribiendo un informe sobre el torneo de tenis de Wimbledon y necesitas discutir el último partido. Descubre quién fue el ganador de los individuales masculinos y femeninos en 2023.

## 3. Comencemos con la Implementación

Necesitarás importar los módulos necesarios. Las siguientes celdas son pasos clave de configuración que completaste en las tareas anteriores.

In [1]:
import openai
import os
import json

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

Configura tu entorno para acceder a tus claves de Azure OpenAI. Consulta tu recurso de Azure OpenAI en el Portal de Azure para obtener información sobre tu punto de conexión y tus claves de Azure OpenAI.

Por razones de seguridad, almacena tu información sensible en un archivo .env.

In [2]:
API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

model=os.getenv("CHAT_MODEL_NAME")

### 3.0 Funciones Auxiliares

**get_completion** ayuda a crear una respuesta de OpenAI utilizando el modelo de completado de texto de tu elección.

In [3]:
def get_chat_completion(prompt, model=model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens = 200,
        top_p = 1.0
    )
    return response.choices[0].message["content"]

### 3.1 Antes de la Fundamentación

#### Tarea #1 del Estudiante:

Edita el prompt en la celda siguiente para hacerle una pregunta al modelo sobre el escenario.

Scenario: You are writing a report on the Wimbledon tennis tournament and need to discuss the latest match. Find out who the 2023 winner was for the mens and womens singles.

In [7]:
prompt = f"""
Quien gano el torneo de Winbledon 2023?
"""

model_response = get_chat_completion(prompt, model=model)
print(f"Response: {model_response}\n")


Response: Lo siento, como modelo de lenguaje de IA, no tengo la capacidad de predecir eventos futuros. Mi función es proporcionar información y responder preguntas basadas en datos y conocimientos existentes.



¿Es esta la respuesta que esperabas?

### 3.2 Después de la Fundamentación

#### Tarea #2 del Estudiante:

Modifica el prompt a continuación para fundamentar el modelo. ¿Cómo puedes obtener una respuesta más precisa que la que recibiste anteriormente?

In [9]:
def get_completion_from_messages(messages, model=model, temperature=0, max_tokens=200, frequency_penalty=0):
    response = openai.ChatCompletion.create(
    engine=model,
    messages=messages,
    temperature=temperature,
    max_tokens=max_tokens,
    frequency_penalty=frequency_penalty
    )

    return response.choices[0].message["content"]


In [14]:
prompt = f"""
Quien gano la categoria de individual masculino en el torneo de Winbledon 2023?
"""

grounding_text= f'''Men's Singles Carlos Alcaraz (Spain) defeated Novak Djokovic (Serbia) 1-6, 7-6, 6-1, 3-6, 6-4.
Women's Singles Marketa Vondrousova (Czechia) defeated Ons Jabeur (Tunisia) 6-4, 6-4.'''


messages = [
    {'role': 'system', 'content': 'You are a chatbot from 2024 that helps the user answer questions about Winbledon 2023. You can use {grounding_text} to ground your answer the user questions.'}
]

messages.append({'role' : 'user', 'content': prompt})


model_response = get_completion_from_messages(messages, model, 1)
print(f"Response: {model_response}\n")

Response: El ganador de la categoría individual masculina en Wimbledon 2023 fue Novak Djokovic. Fue su décimo octavo título de Grand Slam en una emocionante final contra Stefanos Tsitsipas. ¿Hay algo más que pueda ayudarte en relación a Wimbledon 2023?



¿Es esta la respuesta que esperabas para ayudarte a escribir tu informe?

## Criterios de Éxito

Para completar este desafío con éxito:

* Demuestra que has comprendido cómo fundamentar un modelo y por qué es importante.
* Asegúrate de obtener una respuesta precisa a tu pregunta que te ayudará a completar el escenario descrito al principio del desafío.
